In [1]:
!pip install plotly==4.8.0 

In [2]:
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
import regex as re
import zipfile
import gc
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
path_to_archive = r"D:\Papers\Paper 3 - Recommender Systems\archive\\"

projects = pd.read_csv(path_to_archive + 'Projects.csv')
schools = pd.read_csv(path_to_archive + 'Schools.csv')
#teachers = pd.read_csv(path_to_archive + 'Teachers.csv')
donations = pd.read_csv(path_to_archive + 'Donations.csv')
#resources = pd.read_csv('Resources.csv')
donors = pd.read_csv(path_to_archive + 'Donors.csv')

C:\Users\Lenovo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
projects.head(2)

,Project ID,School ID,Teacher ID,Teacher Project Posted Sequence,Project Type,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,NaN


In [5]:
projects.isna().sum()

Project ID                               0
School ID                                0
Teacher ID                               0
Teacher Project Posted Sequence          0
Project Type                             0
Project Title                            6
Project Essay                            1
Project Short Description                3
Project Need Statement                   3
Project Subject Category Tree           29
Project Subject Subcategory Tree        29
Project Grade Level Category             0
Project Resource Category               36
Project Cost                             0
Project Posted Date                      0
Project Expiration Date                 14
Project Current Status                   0
Project Fully Funded Date           283253
dtype: int64

In [6]:
projects = projects.dropna(subset=['Project Title',
                        'Project Essay',
                        'Project ID',
                        'Project Need Statement',
                        'Project Posted Date',
                        'Project Expiration Date',
                        'Project Resource Category',
                        'Project Fully Funded Date'
                        ]).reset_index(drop=True)

In [7]:
assert projects.isna().sum().any() == 0

In [8]:
projects = projects[projects['Project Type'] == 'Teacher-Led'].reset_index(drop=True)

In [9]:
projects[['Project Cost', 'Project Resource Category']].groupby(by='Project Resource Category').sum().sort_values('Project Cost', ascending=False) .head()

,Project Cost
Project Resource Category,
Technology,1.679534e+08
Supplies,1.601576e+08
Books,7.434509e+07
Other,2.961288e+07
Computers & Tablets,2.005473e+07


In [10]:
# Cool, now let's see which project categories are more popular! Which ones are they also costly?
df_exp_3 =  projects[['Project Cost', 'Project Resource Category', 'Project ID']].groupby(by='Project Resource Category').agg({'Project Cost': 'sum', 'Project ID': 'count'}).sort_values(by=('Project ID'))


fig = px.scatter(x = df_exp_3.index,
             y = df_exp_3[('Project ID')],
             size = df_exp_3[('Project Cost')],
             labels=dict(x="Resource Category Type", y="Number of Projects"),
             title="What Projects Have more Popularity and Cost? (radial:cost)")
fig.show()
# love how books are both popular and cheap in comparison to others :)

In [11]:
n_top_categories = 3
important_categories = projects[['Project Cost', 'Project Resource Category', 'Project ID']].groupby(by='Project Resource Category').agg({'Project Cost': 'sum', 'Project ID': 'count'}).sort_values('Project ID', ascending=False) .head(n_top_categories).index.values.tolist()
important_categories

['Supplies', 'Technology', 'Books']

In [12]:
projects = projects[projects['Project Resource Category'].isin(important_categories)].reset_index(drop=True)

In [13]:
assert len(projects) == len(projects.drop_duplicates(subset='Project ID'))

In [14]:
# which grades are more active? Which submit more costly projects?

df_exp_1 = projects[['Project Cost', 'Project ID', 'Project Grade Level Category']].groupby('Project Grade Level Category').agg(['count', 'mean']).reset_index().sort_values('Project Grade Level Category').set_index('Project Grade Level Category')


fig = go.Figure(go.Barpolar(
    theta=df_exp_1.index,\
    r=df_exp_1[('Project Cost', 'count')],
    width=df_exp_1[('Project Cost', 'mean')]/1000,
    marker_color=['gold', 'mediumturquoise', 'darkorange', 'lightgreen', 'pink'],
    marker_line_color="black",
    marker_line_width=2,
    opacity=0.8
))
fig.update_layout(title="How Much each Grade spent on Average? (radial:count, width:Average cost)")

fig.show()

In [15]:
projects = projects[projects['Project Grade Level Category']!='unknown']

In [16]:
assert donations.isna().sum().any() == 0

In [17]:
donors.isna().sum()#.any() == 0

Donor ID                 0
Donor City          213097
Donor State              0
Donor Is Teacher         0
Donor Zip           180060
dtype: int64

In [18]:
len(donors.dropna()) / len(donors)

0.899607564165379

In [19]:
donors.dropna(inplace=True)
donors.reset_index(inplace=True, drop=True)

In [24]:
assert donors.isna().sum().any() == 0

In [20]:
schools.isna().sum()

School ID                          0
School Name                        0
School Metro Type                  0
School Percentage Free Lunch    1141
School State                       0
School Zip                         0
School City                      227
School County                      2
School District                    0
dtype: int64

In [21]:
len(schools.dropna()) / len(schools)

0.9822037729645309

In [22]:
schools = schools.dropna().reset_index(drop=True)
assert schools.isna().sum().any() == 0

In [23]:
merge_1 = pd.merge(projects, donations, on='Project ID', how='inner')
merge_2 = pd.merge(merge_1, schools, on='School ID', how='inner')
merge_3 = pd.merge(merge_2, donors, on='Donor ID', how='inner')

In [24]:
del projects
del donors
del donations
del schools


del df_exp_1
del df_exp_3
del merge_1
del merge_2

In [33]:
gc.collect()

1588

In [25]:
df_exp = merge_3.groupby(by= ['School Metro Type']).agg({'Donation Amount': 'sum', 'Donor ID': 'count'}).sort_values(['Donation Amount', 'Donor ID']).reset_index(drop=False)
df_exp.loc[:, 'donation_amount_per'] = df_exp.loc[:, 'Donation Amount'] / df_exp['Donation Amount'].sum()
df_exp.loc[:, 'donation_freq_per'] = df_exp.loc[:, 'Donor ID'] / df_exp['Donor ID'].sum()

In [26]:

fig = make_subplots(
    rows=1, cols=2,
    #column_widths=[0.6, 0.4],
    #row_heights=[0.4, 0.6],
    #specs=[[{}, {}]])

)

fig.add_trace(
    go.Bar(x = df_exp['School Metro Type'], y = df_exp['Donation Amount'], name = 'Amount donated to each'),
    row=1, col=1
)
df_exp.sort_values(by=('Donation Amount'), inplace=True)
fig.add_trace(
    go.Bar(x = df_exp['School Metro Type'], y = df_exp['Donor ID'], name = 'Number of Donations'),
    row=1, col=2
)

fig.show()


In [27]:
df_exp = merge_3.groupby(['School State']).agg({'Donation Amount': 'sum', 'Donor ID': 'count'})
df_exp.loc[:, 'donation_amount_per'] = df_exp.loc[:, 'Donation Amount'] / df_exp['Donation Amount'].sum()
df_exp.loc[:, 'donation_freq_per'] = df_exp.loc[:, 'Donor ID'] / df_exp['Donor ID'].sum()
df_exp.reset_index(inplace=True)

In [28]:
df_exp.head()

,School State,Donation Amount,Donor ID,donation_amount_per,donation_freq_per
0,Alabama,2168002.43,35846,0.011220,0.011845
1,Alaska,641809.18,8527,0.003322,0.002818
2,Arizona,3530710.12,54186,0.018273,0.017905
3,Arkansas,1239199.22,22157,0.006413,0.007322
4,California,31626628.16,457388,0.163683,0.151139


In [29]:
# not important plot
fig = make_subplots(
    rows=1, cols=2,
    #column_widths=[0.6, 0.4],
    #row_heights=[0.4, 0.6],
    #specs=[[{}, {}]])

)

fig.add_trace(
    go.Bar(x = df_exp['School State'], y = df_exp['Donation Amount'], name = 'Amount donated to each'),
    row=1, col=1
)
df_exp.sort_values(by=('Donation Amount'), inplace=True)
fig.add_trace(
    go.Bar(x = df_exp['School State'], y = df_exp['Donor ID'], name = 'Number of Donations'),
    row=1, col=2
)
fig.update_layout(xaxis_tickangle=-50)

fig.show()


In [48]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

m = 20

df_exp = merge_3.groupby('School State').agg({'Donation Amount': 'sum', 'Donor ID': 'count'})
df_exp.loc[:, 'donation_amount_per'] = df_exp.loc[:, 'Donation Amount'] / df_exp['Donation Amount'].sum()
df_exp.loc[:, 'donation_freq_per'] = df_exp.loc[:, 'Donor ID'] / df_exp['Donor ID'].sum()

df_exp = df_exp.sort_values(['Donation Amount', 'Donor ID']).reset_index(drop=False)
y_saving = df_exp['Donor ID'][-m:]
y_net_worth = df_exp['Donation Amount'][-m:]
x = df_exp['School State'][-m:]


# Creating two subplots
fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=True, vertical_spacing=0.001)

fig.append_trace(go.Bar(
    x=y_saving,
    y=x,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Total Number of Donations',
    orientation='h',
), 1, 1)

fig.append_trace(go.Scatter(
    x=y_net_worth, y=x,
    mode='lines+markers',
    line_color='rgb(128, 0, 128)',
    name='Total Amount of Donation',
), 1, 2)

fig.update_layout(
    title='Comparison between the States',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=False,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=40000,
    ),
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

y_s = np.round(df_exp['donation_freq_per'][-m:], decimals=2)*100
y_nw = np.rint(y_net_worth)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, x):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn - 2000000,
                            text='{:,}'.format(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 400000,
                            text=str(yd) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(50, 171, 96)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=0,
                        font=dict(family='Arial', size=10, color='rgb(150,150,150)'),
                        showarrow=True))

fig.update_layout(annotations=annotations)

fig.show()

In [31]:
df_exp = merge_3[['School State', 'Project Cost', 'Project ID']].groupby(['School State']).agg({'Project Cost': 'mean', 'Project ID': 'count'}).sort_values(by ='Project ID')

fig = px.scatter(x = df_exp.index,
             y = df_exp[('Project ID')],
             size = df_exp[('Project Cost')],
             labels=dict(x="Area Type", y="count"),
             title='projects count and mean cost')
fig.show()


In [32]:
top_10_pro_states = df_exp.sort_values(by ='Project ID', ascending=False).head(10).index.tolist()
top_10_pro_states

['California',
 'New York',
 'Texas',
 'Florida',
 'Illinois',
 'North Carolina',
 'Georgia',
 'Pennsylvania',
 'Michigan',
 'South Carolina']

In [33]:
df_exp = merge_3[['Donor State', 'Donation Amount', 'Donor ID']].groupby('Donor State').agg({'Donor ID': 'count',
                                                                                        'Donation Amount': 'median'}).sort_values(by='Donor ID')

fig = px.scatter(x = df_exp.index,
             y = df_exp['Donor ID'],
             size = df_exp['Donation Amount'],
             labels=dict(x="Donor State", y="count of Donors"),
             title='Number of Donors (size is the median of donations)',
             text = df_exp['Donation Amount'], )

fig.update_traces(texttemplate='%{text:.2s}', textposition='bottom center', textfont_size=8)
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


In [34]:
top_10_donor_states = df_exp.tail(10).index.tolist()
top_10_donor_states

['Michigan',
 'Massachusetts',
 'Georgia',
 'Pennsylvania',
 'North Carolina',
 'Florida',
 'Illinois',
 'Texas',
 'New York',
 'California']

In [35]:
set_10 = set().union(top_10_donor_states + top_10_pro_states)
len(set_10)

11

In [36]:
set_10

{'California',
 'Florida',
 'Georgia',
 'Illinois',
 'Massachusetts',
 'Michigan',
 'New York',
 'North Carolina',
 'Pennsylvania',
 'South Carolina',
 'Texas'}

In [38]:
merge_3 = merge_3[merge_3['School Metro Type'].isin(['suburban', 'urban'])]
merge_3 = merge_3[(merge_3['Donor State'].isin(set_10)) & (merge_3['School State'].isin(set_10))]

merge_3.reset_index(inplace=True, drop=True)

In [39]:
len(merge_3)

1280647

In [40]:
merge_3.drop_duplicates(subset=['Donation ID'], keep='last', inplace=True)
merge_3.reset_index(inplace=True, drop=True)

In [41]:
#

In [42]:
df_exp = merge_3.groupby(by='Project ID'). agg({'Donation ID': 'count', 'Donation Amount':'sum'}).sort_values('Donation ID')
df_exp.reset_index(inplace=True)


pers = []
project_per = []
don_rem = []
m = 50
print('we filter tha projects with more than i donations\nThen we see what percentage of the main df will remain')
for i in range(1,m):
    projects_ = df_exp[df_exp['Donation ID']>i]['Project ID']

    per = len(merge_3[merge_3['Project ID'].isin(projects_)])/len(merge_3)
    don_ = int(per*len(merge_3))
    pers.append(per)
    project_per.append(len(projects_))
    don_rem .append(don_)

we filter tha projects with more than i donations
Then we see what percentage of the main df will remain


In [43]:
df_remaining.to_csv('df_remaining.csv', index=False)

NameError: name 'df_remaining' is not defined

In [ ]:



df_remaining = pd.DataFrame(
    {
     'more than i donations for each project': range(1,m),
     'remaining perc. of donation df': pers,
     'numb. of projects remaining': project_per,
     'number of donations remaining': don_rem
    }
)

fig = px.scatter(x = range(1,m), y=pers, text=pers, hover_data=[''])
fig.update_layout(
    title="Filtering the Dataset",
    xaxis_title="We remove projects with less than x donations",
    yaxis_title="Percentage that Remains",
    legend_title="Legend Title",
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='bottom center', textfont_size=10)
fig.update_layout(hovermode="x unified")

In [70]:
don_count = merge_3[['Donor ID', 'Project ID']].groupby('Project ID').count().sort_values(by= 'Donor ID',ascending = False)
df_exp = don_count.reset_index()
px.histogram(df_exp['Donor ID'])

In [67]:
n_donors_to_each_pro = 3
project_filter_list = don_count[don_count['Donor ID'] >=n_donors_to_each_pro].index.tolist()


In [37]:
len(merge_3[merge_3['Project ID'].isin(project_filter_list)]) / len(merge_3)

NameError: name 'project_filter_list' is not defined